# QUANTUM SUPPORT VECTOR MACHINES

En este notebook, usaremos circuitos cuánticos para calcular un kernel que luego utilizaremos como parte de un SVM. Usaremos la implementación de Qiskit, que ya encapsula el cálculo de la matriz del kernel en el entrenamiento del SVM.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import Aer
from qiskit_machine_learning.datasets import breast_cancer, iris, digits
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.algorithms import QSVC

## Un primer ejemplo

Empezaremos con un ejemplo muy sencillo que es linealmente separable.

In [ ]:
np.random.seed(2021)
n = 30

# Ejemplos de entrenamiento para la clase 0

mean = [0, 1]
cov = [[0.1, 0], [0, 0.1]]

A = np.random.multivariate_normal(mean, cov, n)
x_A, y_A = A.T
plt.plot(x_A, y_A, 'ro');

# Ejemplos de entrenamiento para la clase 1

mean = [1, 0]
cov = [[0.1, 0], [0, 0.1]]

B = (np.random.multivariate_normal(mean, cov, n))
x_B, y_B = B.T
plt.plot(x_B, y_B, 'bo')

training_input = np.append(A,B,axis =0)
training_labels = np.array([0]*n+[1]*n) # Etiqueta 0 para los primeros n ejemplos y 1 para los siguientes 

Como primer circuito para el kernel, usaremos *ZFeatureMap*

In [ ]:
feature_map = ZFeatureMap(feature_dimension=2, reps=1)
feature_map.decompose().draw(output="mpl")

Una vez generados los datos y definido el feature map, podemos entrenar el SVM

In [ ]:
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)
quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
qsvm = QSVC(quantum_kernel = quantum_kernel)
qsvm.fit(training_input,training_labels)
print("QSVC entrenado")

Para comprobar su rendimiento, vamos a generar ejemplos de test siguiendo la misma distribución.

In [ ]:
n = 10

# Ejemplos de test para la clase 0

mean = [0, 1]
cov = [[0.1, 0], [0, 0.1]]

C = (np.random.multivariate_normal(mean, cov, n))
x_C, y_C = C.T

# Ejemplos de test para la clase 1

mean = [1, 0]
cov = [[0.05, 0], [0, 0.05]]

D = (np.random.multivariate_normal(mean, cov, n))
x_D, y_D = D.T


Podemos usar el QSVC entrenado para predecir las etiquetas y, también, calcular la accuracy directamente. 

In [ ]:
print("Predicción para los ejemplos de la clase 0",qsvm.predict(C))
print("Predicción para los ejemplos de la clase 1",qsvm.predict(D))

test_input = np.append(C,D,axis =0)
test_labels = np.array([0]*n+[1]*n)

print("Accuracy",qsvm.score(test_input,test_labels))

# Ejemplos con otros datasets 

Qiskit incorpora algunos datasets sencillos que podemos usar para hacer pruebas

In [ ]:
training_input, training_labels, test_input, test_labels = breast_cancer(
    training_size=100,
    test_size=20,
    n=2,
    plot_data=True,
    one_hot = False,
)

Usamos primero un feature map sencillo

In [ ]:
feature_map = ZFeatureMap(feature_dimension=2, reps=1)
quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
qsvm = QSVC(quantum_kernel = quantum_kernel)
qsvm.fit(training_input,training_labels)
print("Accuracy", qsvm.score(test_input,test_labels))

Ahora, un feature map más complejo

In [ ]:
feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement='linear')
feature_map.decompose().draw(output="mpl")

In [ ]:
quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
qsvm = QSVC(quantum_kernel = quantum_kernel)
qsvm.fit(training_input,training_labels)
print("Accuracy", qsvm.score(test_input,test_labels))

También podemos hacer clasificación multiclase. QSCV extiende a la clase SVC de scikit-learn, así que gestiona automáticamente la clasificación multiclase (con el método one-vs-one).

In [ ]:
training_input, training_labels, test_input, test_labels = iris(
    training_size=100,
    test_size=20,
    n=2,
    plot_data=True,
    one_hot = False,
)

In [ ]:
feature_map = ZZFeatureMap(feature_dimension=2, reps=1, entanglement='linear')
quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
qsvm = QSVC(quantum_kernel = quantum_kernel)
qsvm.fit(training_input,training_labels)
print("Accuracy",qsvm.score(test_input,test_labels))